# [모듈 2.1] CodeBuild 프로젝트 생성

- 개발자 가이드
    - [Create a build project (AWS CLI)](https://docs.aws.amazon.com/codebuild/latest/userguide/create-project-cli.html)
    

aws codebuild list-projects --sort-by NAME --sort-order ASCENDING    
aws codebuild batch-get-projects --names iris-build-project    

# 1. 환경 설정


## 1.1. 파라미터 설정

In [ ]:
import sagemaker
import boto3

region = sagemaker.Session().boto_region_name

account_id = boto3.client('sts').get_caller_identity().get('Account')


print("region: ", region)
print("account_id: ", account_id)

In [ ]:
isDefaultBucket = True

if isDefaultBucket:

    bucket = sagemaker.Session().default_bucket()
else:
    bucket = "<Type your bucket name>"
    
print("Bucket: ", bucket)    


## 1.2. 역할 생성 및 정의
### 아래 Code Build 역할 ARN 을 넣어 주세요
- build_service_role: 은 아래와 미리 생성이 됨
    -  "arn:aws:iam::XXXXXX:role/codebuild-gsmoon"

In [ ]:
code_build_project_name = "Hello_Code_Build"
build_service_role = "arn:aws:iam::XXXXXXX:role/MLOPS-CodeBuildRole-1BZ6X57P61FS1"

# 2. 빌드 프로젝트  JSON 정의
- 

In [ ]:
build_dic = {
  "name": code_build_project_name,
  "description": "Hello Build Project",
  "source": {
    "type": "CODEPIPELINE",
    "buildspec": "buildspec.yml",
  },
  "artifacts": {
    "type": "CODEPIPELINE",
    "name": code_build_project_name,
  },
  "cache": {
    "type": "NO_CACHE",
  },
  "environment": {
            "type": "LINUX_CONTAINER",
            "image": "aws/codebuild/amazonlinux2-x86_64-standard:3.0",
            "computeType": "BUILD_GENERAL1_SMALL",
            "environmentVariables": [
                {
                    "name": "IMAGE_REPO_NAME",
                    "value": "iris-model",
                    "type": "PLAINTEXT"
                },
                {
                    "name": "IMAGE_TAG",
                    "value": "latest",
                    "type": "PLAINTEXT"
                },
                {
                    "name": "AWS_ACCOUNT_ID",
                    "value": account_id,
                    "type": "PLAINTEXT"
                },
                {
                    "name": "AWS_DEFAULT_REGION",
                    "value": region,
                    "type": "PLAINTEXT"
                },
                {
                    "name": "TEMPLATE_BUCKET",
                    "value": bucket,
                    "type": "PLAINTEXT"
                },
                {
                    "name": "TEMPLATE_PREFIX",
                    "value": "codebuild",
                    "type": "PLAINTEXT"
                }
            ],
            "privilegedMode": False,
            "imagePullCredentialsType": "CODEBUILD"
        },
  "serviceRole": build_service_role,
  "timeoutInMinutes": 60,
  "queuedTimeoutInMinutes": 480,
  "badgeEnabled": False,
  "logsConfig": {
    "cloudWatchLogs": {
      "status": "ENABLED",
    },
    "s3Logs": {
      "status": "DISABLED",
      "encryptionDisabled": False
    }
  },
}

## 2.1. Dic 포맷을 JSON 으로 변경

In [ ]:
import json

build_json = json.dumps(build_dic)
json.loads(build_json)

## 2.2. JSON 파일 저장

In [ ]:
import os

os.makedirs('src',exist_ok=True)

In [ ]:
json_file_path = 'src/hello_build.json'
with open(json_file_path, "w") as outfile:
    outfile.write(build_json)

In [ ]:
with open(json_file_path) as json_file:
    json_data = json.load(json_file)
    
json_data    

# 3. 빌드 프로젝트 생성

In [ ]:
%%sh -s {json_file_path}
json_file_path=$1
echo $json_file_path
aws codebuild create-project --cli-input-json file://$json_file_path

## 참고: 코드 빌드 프로젝트 리스트
- [Boto3 ListProjects](https://docs.aws.amazon.com/codebuild/latest/APIReference/API_ListProjects.html)

In [ ]:
%%sh 
aws codebuild list-projects --sort-by LAST_MODIFIED_TIME --sort-order DESCENDING

# 4. 프로젝트 이름 저장

In [ ]:
%store code_build_project_name
%store account_id
%store region
%store bucket
